# Uniref mappings



In [1]:
import org.apache.spark._
import org.apache.spark.sql.types._
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._
import kernel.display.html

In [2]:
val whalePath = "/data/results/gray-whale/"
val expressionsPath = whalePath + "Expressions/"
val unirefPath = expressionsPath + "uniref90/"
val transcriptsPath = expressionsPath + "Transcripts/"
val codingPath = transcriptsPath + "coding/"

val comparisonsPath = expressionsPath + "Comparisons/"
val comparisonsUniref = comparisonsPath + "uniref90_comparisons/"
val annotationsPath = comparisonsPath + "annotations/"

In [3]:
def loadTranscripts(subpath: String, prefix: String) = {
    val path = if(subpath.startsWith("/")) subpath else transcriptsPath + subpath
    spark.readTSV(path, header=true).select($"Name".as("transcript"), $"NumReads".as(prefix + "_reads"), $"TPM".as(prefix + "_TPM")).cache 
}

In [4]:
import org.apache.spark.sql.functions.udf
def undot(str: String): String = str.substring(0, str.indexOf("."))
def uni(df: DataFrame) = df.select("uniref90").distinct
val undotFun = udf[String, String](undot)

In [5]:
val gray_liver_tr = loadTranscripts("raw/gray_whale/liver/transcripts_quant", "gray_whale_liver")
val gray_kidney_tr =  loadTranscripts("raw/gray_whale/kidney/transcripts_quant", "gray_whale_kidney")
(gray_liver_tr.count, gray_kidney_tr.count)

(114263,114263)

In [6]:
val anno = spark.readTSV(comparisonsPath + "annotations/annotations_uniref.tsv", header = true)
anno.limit(10).show(10,1000)

+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------

In [7]:
val mapping = spark.readTSV("/data/indexes/uniprot/idmapping_selected.tab").toDF("UniProtKB-AC","UniProtKB-ID","Entrez","RefSeq","GI","PDB","GO",
"UniRef100","UniRef90","UniRef50","UniParc","PIR",
"NCBI-taxon","MIM","UniGene","PubMed",
"EMBL","EMBL-CDS","Ensembl","Ensembl_TRS","Ensembl_PRO","Additional PubMed"
    )
mapping.limit(20).show(20, 1000)

+------------+------------+-------+-----------+-------------------------------+----+----------------------------------+----------------+---------------+---------------+-------------+----+----------+----+-------+------------------+--------+----------+-------+-----------+-----------+-----------------+
|UniProtKB-AC|UniProtKB-ID| Entrez|     RefSeq|                             GI| PDB|                                GO|       UniRef100|       UniRef90|       UniRef50|      UniParc| PIR|NCBI-taxon| MIM|UniGene|            PubMed|    EMBL|  EMBL-CDS|Ensembl|Ensembl_TRS|Ensembl_PRO|Additional PubMed|
+------------+------------+-------+-----------+-------------------------------+----+----------------------------------+----------------+---------------+---------------+-------------+----+----------+----+-------+------------------+--------+----------+-------+-----------+-----------+-----------------+
|      Q6GZX4|  001R_FRG3G|2947773|YP_031579.1|             81941549; 49237298|null|             

### Load Bat expressions



In [16]:
val quants_base = "/data/samples/de_novo/bat/quants/ours"
val bat_liver_1 = spark.readTSV(quants_base + "/quant_bat_liver_active_1/quant.sf", header = true)
val bat_kidney_1 = spark.readTSV(quants_base + "/quant_bat_kidney_active_1/quant.sf", header = true)
val bat_liver_2 = spark.readTSV(quants_base + "/quant_bat_liver_active_2/quant.sf", header = true)
val bat_kidney_2 = spark.readTSV(quants_base + "/quant_bat_kidney_active_2/quant.sf", header = true)
bat_liver_1.show(10,10000)

+-----------------------------------------+------+---------------+---------+--------+
|                                     Name|Length|EffectiveLength|      TPM|NumReads|
+-----------------------------------------+------+---------------+---------+--------+
| NODE_1_length_17988_cov_580.222903_g0_i0| 17988|      25996.137| 2.849032|2131.249|
|  NODE_2_length_17947_cov_75.308981_g1_i0| 17947|      23591.296| 5.768584|3916.057|
|  NODE_3_length_17149_cov_48.936455_g2_i0| 17149|      16480.623| 2.802971|1329.291|
| NODE_4_length_16953_cov_143.141987_g3_i0| 16953|      16648.804|11.308167|5417.553|
|  NODE_5_length_16927_cov_28.452499_g4_i0| 16927|      15432.613| 1.503753| 667.796|
|  NODE_6_length_16768_cov_80.455725_g5_i0| 16768|      22827.061| 0.206226| 135.463|
|  NODE_7_length_16714_cov_42.414072_g6_i0| 16714|       13824.43| 1.240009| 493.287|
|  NODE_8_length_16515_cov_85.093674_g5_i1| 16515|      22310.009| 0.971752| 623.855|
|  NODE_9_length_16383_cov_85.107160_g5_i2| 16383|    

### Protein search auxiliary classes




Computing bat mapping<br>

In [19]:
import org.bdgenomics.adam.rdd.ADAMContext._
val bat_protein = spark.sparkContext.loadFastaProtein("/data/results/graywhale/transcriptome/diamond/blastx/transdecoder/miotis_brandis_uniref90/transcripts.fasta.transdecoder.pep")
bat_protein.dataset.show(10,10000)

+--------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
val bat_liver_kidney = bat_liver_1.select($"Name".as("transcript"), $"TPM".as("bat_liver_1"))
  .join(bat_kidney_1.select($"Name".as("transcript"), $"TPM".as("bat_kidney_1")), Seq("transcript"), "outer")
  .join(bat_liver_2.select($"Name".as("transcript"), $"TPM".as("bat_liver_2")), Seq("transcript"), "outer")
  .join(bat_kidney_2.select($"Name".as("transcript"), $"TPM".as("bat_kidney_2")), Seq("transcript"), "outer")

bat_liver_kidney.show(10,1000)

+----------------------------------------------+-----------+------------+-----------+------------+
|                                    transcript|bat_liver_1|bat_kidney_1|bat_liver_2|bat_kidney_2|
+----------------------------------------------+-----------+------------+-----------+------------+
| NODE_100234_length_469_cov_1.373239_g72932_i0|        0.0|    0.191479|        0.0|    0.383722|
| NODE_101117_length_465_cov_2.876777_g73808_i0|   0.187845|    0.282301|    0.29153|    0.517952|
| NODE_101326_length_464_cov_3.648456_g74016_i0|   0.111337|    0.628155|   0.162481|    0.231963|
| NODE_101327_length_464_cov_3.579572_g74017_i0|    0.26961|     0.28013|   0.299512|    0.171259|
| NODE_101547_length_463_cov_4.180952_g74235_i0|   0.123989|    0.624901|        0.0|     0.32064|
|NODE_101694_length_462_cov_29.210024_g74381_i0|   8.171761|         0.0|   3.018185|         0.0|
| NODE_101740_length_462_cov_5.133652_g74427_i0|   1.489245|    0.213611|   0.385248|    0.983165|
| NODE_101

In [21]:
def un_protein = udf[String, String](str=> str.substring(0, str.indexOf(".p")))
def to_orf = udf[Array[String], String]{ str => 
 val of = str.substring(str.indexOf("ORF")+4)
 of.substring(0, of.indexOf("NODE")-1).split(',')
}
//def separate = udf[Array[String], String](str => str.split(','))

In [22]:
import scala.collection.mutable.WrappedArray
def get_array(num: Int): UserDefinedFunction = udf[String, WrappedArray[String]](arr => arr(num))
val orf_type = get_array(0)
val orf_score =   udf[Double, WrappedArray[String]](arr => arr(1).replace("score=", "").toDouble)

case class UnirefScore(uniref: String, score: Double, p: String)
def uniref = udf[WrappedArray[UnirefScore], WrappedArray[String]]{case arr => 
    val ars = arr.tail.tail
    ars.map{ case a => 
        val u = a.split('|')
        UnirefScore(u(0), u(1).toDouble, u(2))
    }
    }

def len = udf[Int, String]{ case str=> 
    val l = str.indexOf("len:")
    str.substring(l + 4, str.indexOf(" (")).toInt
}

val bat_protein_dataset = bat_protein.dataset
.withColumn("transcript", un_protein($"name"))
.withColumn("orf",  to_orf($"description"))
.withColumn("orf_type", orf_type($"orf"))
.withColumn("orf_len",  len($"orf_type"))
.withColumn("orf_score", orf_score($"orf"))
.withColumn("uniref", uniref($"orf"))
.select("transcript", "orf_type", "orf_len", "orf_score",  "uniref")
.as[(String, String, Int, Double, scala.collection.mutable.ArrayBuffer[UnirefScore])]
.filter(r=>r._5.size != 0 && r._4 > 0 && r._3 > 100)
bat_protein_dataset.show(10,10000)

+-----------------------------------------------+-------------------------------+-------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
bat_protein_dataset.count

48054

In [24]:
import org.apache.spark.sql.Row
def best_hit_uniref = udf[String,WrappedArray[Row]]{arr=>
    val row = arr.maxBy(r=>r.getDouble(1))
    row.getString(0)
}
def best_hit_score = udf[Double,WrappedArray[Row]]{arr=>
    val row = arr.maxBy(r=>r.getDouble(1))
    row.getDouble(1)
}

//UniRef90_G1PK73, 93.9 , UniRef90_G1P3P3, 100.0
//UniRef90_UPI000767D85D, 95.6
val best_bat = bat_protein_dataset
.withColumn("best_hit_score", best_hit_score($"uniref"))
.withColumn("uniref90", best_hit_uniref($"uniref"))
.select("uniref90","best_hit_score","transcript", "orf_type", "orf_score")
best_bat.show(10, 1000)


+----------------------+--------------+-----------------------------------------------+-------------------------------+---------+
|              uniref90|best_hit_score|                                     transcript|                       orf_type|orf_score|
+----------------------+--------------+-----------------------------------------------+-------------------------------+---------+
|       UniRef90_G1PK73|          93.9| NODE_142717_length_344_cov_1.757475_g115348_i0|      type:internal len:115 (+)|    25.85|
|UniRef90_UPI000767D85D|          95.6|  NODE_7016_length_4014_cov_135.133719_g3475_i0|     type:complete len:1088 (+)|   221.73|
|       UniRef90_G1NV97|          99.0|   NODE_33731_length_1474_cov_7.901468_g7328_i1|      type:complete len:103 (+)|    22.43|
|UniRef90_UPI000CCC3863|          87.1|NODE_56112_length_848_cov_962.957764_g17657_i16|      type:complete len:244 (-)|    80.78|
|       UniRef90_S7PJX8|          99.8|   NODE_7371_length_3923_cov_45.125515_g1620_i2|   

In [25]:
val bat_mappings = best_bat.join(bat_liver_kidney , Seq("transcript"))
bat_mappings.show(10,1000)

+------------------------------------------------+----------------------+--------------+-------------------------------+---------+-----------+------------+-----------+------------+
|                                      transcript|              uniref90|best_hit_score|                       orf_type|orf_score|bat_liver_1|bat_kidney_1|bat_liver_2|bat_kidney_2|
+------------------------------------------------+----------------------+--------------+-------------------------------+---------+-----------+------------+-----------+------------+
|   NODE_101874_length_462_cov_1.926014_g74561_i0|       UniRef90_S7NS67|         100.0|type:5prime_partial len:135 (+)|    12.29|   0.672502|    0.580052|   0.496259|    0.119216|
|   NODE_10277_length_3384_cov_126.237055_g742_i2|       UniRef90_G1PWG9|          97.2|      type:complete len:575 (+)|    99.66|        0.0|    1.599671|    0.07182|    0.372074|
|   NODE_103361_length_456_cov_2.174334_g76040_i0|       UniRef90_S7MIG1|          75.0|type:5p

In [26]:
(best_bat.count, best_bat.select("uniref90").distinct.count, best_bat.select("transcript").distinct.count, best_bat.select("transcript", "uniref90").distinct.count)

(48054,28068,41691,47121)

In [43]:
bat_mappings.count()

48054

In [27]:
//bat_mappings.writeTSV("/data/results/gray-whale/mappings/bat_mappings.tsv", header = true)
//bat_mappings.writeParquet("/data/results/gray-whale/best_bap_mappings.parquet", local = true)
val bat_mappings = spark.readTSV("/data/results/gray-whale/mappings/bat_mappings.tsv", header = true) //spark.readParquet("/data/results/gray-whale/best_bap_mappings.parquet")
//bat_mappings.show(10,1000)

In [44]:
bat_mappings.count()

48054

In [28]:
import org.apache.spark.sql.functions._
val bat_unirefs = bat_mappings.select($"uniref90".as("UniRef90"),$"bat_liver_1",$"bat_kidney_1",$"bat_liver_2",$"bat_kidney_2")
.groupBy("UniRef90").agg(
    sum($"bat_liver_1").as("bat_liver_1"), 
    sum($"bat_kidney_1").as("bat_kidney_1"), 
    sum($"bat_liver_2").as("bat_liver_2"), 
    sum($"bat_kidney_2").as("bat_kidney_2")
    ).orderBy($"bat_liver_1".desc, $"bat_kidney_1".desc)
bat_unirefs.show(10,1000)

+----------------------+------------------+-----------------+------------------+------------------+
|              UniRef90|       bat_liver_1|     bat_kidney_1|       bat_liver_2|      bat_kidney_2|
+----------------------+------------------+-----------------+------------------+------------------+
|       UniRef90_S7PWY9|19758.199470000003|         5.516879|15451.375791999999|          8.752093|
|       UniRef90_G1PM73|      19716.186889|         5.006213|      19646.758581|          9.586184|
|       UniRef90_S7NKH8|13639.597238000002|         8.370806|      14385.655255|         12.855774|
|       UniRef90_L5LNJ6|13502.713297000002|        14.285188|15303.831831000001|         13.945957|
|       UniRef90_S7MRF2|       8993.596224|         5.323449|       8062.678561|          5.772969|
|       UniRef90_S7MHE7|       6585.148337|         1.531405|       5983.654563|          2.328961|
|UniRef90_UPI000CCC3436|       6545.906132|         3.378611| 6035.026793999999|          4.432763|


In [29]:
import group.research.aging.spark.extensions.functions.ConcatenateString
val con = new ConcatenateString(";")
val partial_bat_uniref_mappings = mapping.select("UniRef90", "GO").join(bat_unirefs.select("UniRef90"), "UniRef90").distinct.na.fill("")
.groupBy($"UniRef90").agg(con($"GO").as("GO")).cache
partial_bat_uniref_mappings.show(10,1000)

+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|           UniRef90|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [30]:
(partial_bat_uniref_mappings.count, partial_bat_uniref_mappings.select("UniRef90").distinct.count, bat_unirefs.count,bat_unirefs.select("UniRef90").distinct.count)

(14999,14999,28068,28068)

In [31]:
(partial_bat_uniref_mappings.count, partial_bat_uniref_mappings.select("UniRef90").distinct.count, bat_unirefs.count,bat_unirefs.select("UniRef90").distinct.count)

(19767,19767,28268,28268)

In [32]:
def flatGo = udf[Array[String], String]{ str => str.trim.replaceAll("^;", "".replaceAll(" +", "")).split(";")}
val bat_go_mappings = partial_bat_uniref_mappings
.filter($"GO" =!=";")
.withColumn("GO", flatGo($"GO"))
.join(bat_unirefs, Seq("Uniref90"))
.as[(String, scala.collection.mutable.ArrayBuffer[String], Double, Double, Double, Double)]
.flatMap{ case (a, arr, l1, k1, l2, k2) => arr.toList.filter(f=>f.trim!="").distinct.map(aa=> (aa, l1, k1, l2, k2 ))}
.toDF("GO", "bat_liver_1", "bat_kidney_1", "bat_liver_2", "bat_kidney_2")
.groupBy($"GO").agg(sum($"bat_liver_1").as("bat_liver_1"), sum($"bat_kidney_1").as("bat_kidney_1"), sum($"bat_liver_2").as("bat_liver_2"), sum($"bat_kidney_2").as("bat_kidney_2"))
.orderBy($"bat_liver_1".desc,$"bat_kidney_1".desc)
.withColumnRenamed("GO", "go")
bat_go_mappings.show(10,100)

+-----------+------------------+------------------+------------------+------------------+
|         go|       bat_liver_1|      bat_kidney_1|       bat_liver_2|      bat_kidney_2|
+-----------+------------------+------------------+------------------+------------------+
| GO:0005615| 91336.02574200001| 3421.090592000001| 90348.99755000009|3687.9415869999993|
| GO:0005737| 35870.69648499997|25779.108638999987| 34057.78138999998| 25298.75620900001|
| GO:0016021|31293.667532000018| 35415.92437499999|26379.918117000005| 36627.32599800002|
| GO:0016020|28317.733255000006| 21595.71151800001| 27932.64955000001|      20759.532237|
| GO:0005634|22345.484704000002|29520.300637000008|20232.025698000016|28844.954576000004|
| GO:0005829|20550.202911000008|21499.736783999993| 18300.47772899999|21683.063729999994|
| GO:0005576|20491.114166999992| 6261.713387999997| 19631.34355300001| 5706.305368999997|
| GO:0005829|17416.220077999995|      24845.416096| 16745.21679700001| 23481.33331900002|
| GO:00055

In [46]:
(bat_go_mappings.count, bat_go_mappings.select("go").count, bat_go_mappings.select("go").distinct.count)

(13694,13694,13694)

In [45]:
(bat_go_mappings.count, bat_go_mappings.select("go").count, bat_go_mappings.select("go").distinct.count)

(13695,13695,13695)

In [33]:
bat_go_mappings.writeTSV("/data/results/gray-whale/Expressions/GO/raw/bat_go_all.tab")

parts of /data/results/gray-whale/Expressions/GO/raw/bat_go_all.tab merged!


/data/results/gray-whale/Expressions/GO/raw/bat_go_all.tab

In [34]:
partial_bat_uniref_mappings.show(10,100)

+-------------------+-----------------------------------------------------------------------------------+
|           UniRef90|                                                                                 GO|
+-------------------+-----------------------------------------------------------------------------------+
|UniRef90_A0A091DKP3|                         GO:0009008; GO:0008270; GO:0010468 GO:0009008; GO:0010468 |
|UniRef90_A0A146XKR9|                                                                        GO:0016021 |
|UniRef90_A0A1D5QIV1|                                                                                   |
|UniRef90_A0A1D5QT67|             GO:0046875; GO:1901187 GO:0005829; GO:0005654; GO:0046875; GO:1901187 |
|UniRef90_A0A212CZ27|                                                                                   |
|UniRef90_A0A287BL13|                        GO:0005783; GO:0005506; GO:0031418; GO:0016702; GO:0004656 |
|UniRef90_A0A2J8QDC0|GO:0005615; GO:0005509; G

In [35]:
val bat_unis = bat_unirefs.select("UniRef90").distinct


(28268,28268)

In [36]:
val bat_mapped_uniref90 = bat_unirefs.join(mapping, Seq("Uniref90"))
bat_mapped_uniref90.show(20,1000)

+---------------+-----------+-----------------+-----------+------------+------------+------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------

In [37]:
mapping.show(10,10000)

+------------+------------+-------+-----------+-------------------------------+----+----------------------+----------------+---------------+---------------+-------------+----+----------+----+-------+------------------+--------+----------+-------+-----------+-----------+-----------------+
|UniProtKB-AC|UniProtKB-ID| Entrez|     RefSeq|                             GI| PDB|                    GO|       UniRef100|       UniRef90|       UniRef50|      UniParc| PIR|NCBI-taxon| MIM|UniGene|            PubMed|    EMBL|  EMBL-CDS|Ensembl|Ensembl_TRS|Ensembl_PRO|Additional PubMed|
+------------+------------+-------+-----------+-------------------------------+----+----------------------+----------------+---------------+---------------+-------------+----+----------+----+-------+------------------+--------+----------+-------+-----------+-----------+-----------------+
|      Q6GZX4|  001R_FRG3G|2947773|YP_031579.1|             81941549; 49237298|null|            GO:0046782|UniRef100_Q6GZX4|UniRef90_

In [38]:
bat_unirefs.count()

28268

In [39]:
val bat_predictions = spark.readTSV("/data/results/gray-whale/diamond/blastx/transdecoder/miotis_brandis_uniref90/transcripts.fasta.transdecoder.pep")
bat_predictions.show(20,1000)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
//val bat_mapping = mapping.where($"NCBI-taxon" === "2516278").cache()
//bat_mapping.writeTSV("/data/indexes/uniprot/species/bat_mapping.tsv", header = true)
//bat_mapping.show(10,1000)

+------------+----------------+------+------+----+----+----+--------------------+---------------+---------------+-------------+----+----------+----+-------+------+--------+----------+-------+-----------+-----------+-----------------+
|UniProtKB-AC|    UniProtKB-ID|Entrez|RefSeq|  GI| PDB|  GO|           UniRef100|       UniRef90|       UniRef50|      UniParc| PIR|NCBI-taxon| MIM|UniGene|PubMed|    EMBL|  EMBL-CDS|Ensembl|Ensembl_TRS|Ensembl_PRO|Additional PubMed|
+------------+----------------+------+------+----+----+----+--------------------+---------------+---------------+-------------+----+----------+----+-------+------+--------+----------+-------+-----------+-----------+-----------------+
|  A0A411H915|A0A411H915_MYOBR|  null|  null|null|null|null|UniRef100_A0A411H915|UniRef90_C6EUP8|UniRef50_E2DZY0|UPI001024592C|null|   2516278|null|   null|  null|MH328040|QBB19999.1|   null|       null|       null|             null|
|  A0A411H922|A0A411H922_MYOBR|  null|  null|null|null|null|UniR

In [41]:
val anage = spark.readTSV("/data/databases/anage/anage_data.tsv", header = true)
anage.show(10,100)

+------+--------+----------+------------+-----------+--------------+-------------+------------+-----------------------+----------------------+--------------------+---------------------------+--------------+------------------+-------------------------+--------------------------------+----------------+------------------+----------------+--------------------+-----------------------+------+---------------+-----------+------------+------------+----------+------------------+-------------+---------------+------------------------------------------------------------------------+
|HAGRID| Kingdom|    Phylum|       Class|      Order|        Family|        Genus|     Species|            Common name|Female maturity (days)|Male maturity (days)|Gestation/Incubation (days)|Weaning (days)|Litter/Clutch size|Litters/Clutches per year|Inter-litter/Interbirth interval|Birth weight (g)|Weaning weight (g)|Adult weight (g)|Growth rate (1/days)|Maximum longevity (yrs)|Source|Specimen origin|Sample size|Data q

In [42]:
anage.where($"Class" === "Aves").select("Species", "Common name", "Maximum longevity (yrs)").orderBy($"Maximum longevity (yrs)").show(100,1000)

+---------------+------------------------------------------------+-----------------------+
|        Species|                                     Common name|Maximum longevity (yrs)|
+---------------+------------------------------------------------+-----------------------+
|        atratus|                                      Black swan|                   null|
|          vauxi|                                    Vaux's swift|                   null|
|       pipixcan|                                 Franklin's gull|                   null|
|    fuscicollis|                          White-rumped sandpiper|                   null|
|      melanotos|                              Pectoral sandpiper|                   null|
|    ptilocnemis|                                  Rock sandpiper|                   null|
|       flavipes|                               Lesser yellowlegs|                   null|
|   lamelligerus|                                African openbill|                   null|